<a href="https://colab.research.google.com/github/kuriatsu/learning_genetic_algorithms/blob/main/ga_2023rwdc_shokki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install deap

# 必要なモジュールのインポート  
遺伝的アルゴリズムの計算には、DEAPを使用  
pprintは辞書型のprintをファンシーにしてくれる

* 公式ドキュメント
https://deap.readthedocs.io/en/master/index.html  
* 日本語説明ブログ
https://dse-souken.com/2021/05/25/ai-19/  
* Github
https://github.com/DEAP/deap/tree/60913c5543abf8318ddce0492e8ffcdf37974d86

In [107]:
import random
import numpy as np
from pprint import pprint

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

# 変数定義
DAY : 計画期間（30日間の配送拠点を計画する）
LIFT_NUM : リフトの数

リフトのデータベース  
* id : リフトID
* base : 拠点
* capacity : 燃料タンク容量[L]
* cosumption : 計画期間内のリフト消費量予測(リフト03 : １日3L消費)
* initial_remaining : 計画期間初日の燃料残量
* remaining : N日目の燃料残量、GA計算中にN日目の燃料残量をシミュレーションするために使用する変数

In [122]:
DAY=30

lifts = [
  {
    "id":"03",
    "base": "kariya",
    "capacity": 60,
    "consumption": [3]*DAY,
    "initial_remaining": 60,
    "remaining": 60,
  },
  {
    "id":"30",
    "base": "takahama",
    "capacity": 60,
    "consumption": [2]*DAY,
    "initial_remaining": 40,
    "remaining": 40,
  },
  {
    "id":"52",
    "base": "takahama",
    "capacity": 60,
    "consumption": [1]*DAY,
    "initial_remaining": 10,
    "remaining": 10,
  },
  {
    "id":"19",
    "base": "higashiura",
    "capacity": 60,
    "consumption": [2]*DAY,
    "initial_remaining": 30,
    "remaining": 30,
  },
  {
    "id":"32",
    "base": "higashiura",
    "capacity": 60,
    "consumption": [1]*DAY,
    "initial_remaining": 10,
    "remaining": 10,
  },
]

LIFT_NUM=len(lifts)

# DEAPモデル構築  
遺伝的アルゴリズムの肝となる、親世代から子世代への遺伝方法は、いくつか選択肢があるので、調査してよりよいものを試してみてください
* トーナメント方式の選択肢 : https://github.com/DEAP/deap/blob/master/deap/tools/selection.py
* 交叉関数の選択肢 : https://github.com/DEAP/deap/blob/master/deap/tools/crossover.py
* 突然変異関数の選択肢 : https://github.com/DEAP/deap/blob/master/deap/tools/mutation.py

In [118]:
## 最小化問題として定義するので、-1.0を与える
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
## 1計画サンプルの定義、リスト内に30日分の配送計画が格納される
creator.create("Individual", list, fitness=creator.FitnessMin)

## GAに必要な各種関数の定義
toolbox = base.Toolbox()
## 1日の配送計画の値の生成方法(各リフトへの給油の組み合わせ=2^LIFT_NUM)
toolbox.register("attribute", random.randint, 0, 2**LIFT_NUM)
## 各サンプルに含まれる30日分の計画をattributeによって決定することを登録
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, DAY)
## サンプル集団の個体数を決定するための関数
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

## トーナメント方式で次世代に子を残す親を選択, サンプル集団内で上から5番めまでのサンプルを次の世代に残す
toolbox.register("select", tools.selTournament, tournsize=5)
## 交叉関数を設定。2点交叉を採用
toolbox.register("mate", tools.cxTwoPoint)
## 突然変異関数の設定。0.2の確率で次の世代で突然変異を起こす。変異の幅(low-up)はサンプルが取りうる値の幅
toolbox.register("mutate", tools.mutUniformInt, low=0, up=2**LIFT_NUM, indpb=0.2)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


# 各サンプルの評価関数
サンプルの内約  
```python
[0, 1, 10, 0, ...]
 |-> 00000 = 1日目、どこにも配送しない
    |-> 00001 = 2日目、"lifts"の0番目のリフト(03)へ給油
       |-> 01010 = 3日目、"lifts"の1と3番目のリフトへ給油
```

In [126]:
def evalPlan(individual):
  """各サンプルの評価関数
  ~args~
  individual : １サンプル
  ~returns~
  reward : 評価関数の値(末尾に,が必要らしい)
  """
  reward = 0 ## この値を最小化

  ## リフトの残燃料を初期化
  for lift_index, lift in enumerate(lifts):
    lift["remaining"] = lift["initial_remaining"]

  ## 配送とリフトの残燃料をシミュレート
  for day, supply_plan_in_a_day in enumerate(individual):
    delivery = set() ## １日の配送拠点
    time = 0 ## 配送にかかった時間[min] (まだ何にも使用してない)

    for lift_index, lift in enumerate(lifts):
      ## 残燃料が２日分以下になったらペナルティ
      if (day+1 < DAY-1 and lift["capacity"] - lift["consumption"][day] - lift["consumption"][day+1] < 0) or \
         (day > 0 and lift["capacity"] - lift["consumption"][day-1] - lift["consumption"][day] < 0):
        reward += 1000

      ## 予測に基づいて、燃料を消費
      lift["remaining"] -= lift["consumption"][day]

      ## day日目の給油計画にlift番目のリフトが存在するかチェック(ビット演算)
      if supply_plan_in_a_day & (1<<lift_index):
        delivery.add(lift["base"]) ## 給油拠点をリストに追加
        time += 0.03 * (lift["capacity"] - lift["remaining"]) ## 給油時間を計算
        lift["remaining"] = lift["capacity"] ## 燃料満タン

    ## 1日の配送拠点に対するペナルティ
    for delivery_base in delivery:
      reward += 10 # 拠点へ配送した際のペナルティ(適当)
      time += 90 # 配送時間90分(適当)

  return reward,

toolbox.register("evaluate", evalPlan)

# 最適化開始
パレートフロントを解としているが、多目的最適化問題のための関数な気がする。。。

In [ ]:
random.seed(64)

pop = toolbox.population(n=10000)
hof = tools.ParetoFront()
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg    	std    	min	max
0  	10000 	582.103	46.9456	390	760
1  	6009  	529.899	37.2634	390	680
2  	6130  	489.902	35.2447	330	630
3  	6089  	453.862	34.5743	300	610
4  	6006  	420.975	34.3654	290	630
5  	6006  	390.373	34.8076	270	570
6  	6122  	361.385	35.4713	240	560
7  	5991  	333.15 	36.5754	220	520
8  	5928  	306.404	37.8419	180	500
9  	6000  	280.611	38.5048	150	470
10 	6085  	256.307	39.876 	120	480
11 	5959  	231.428	40.5623	110	500
12 	6080  	207.638	42.8269	80 	430
13 	6024  	184.303	44.2275	60 	410
14 	5939  	161.911	45.4295	60 	440
15 	5963  	140.422	46.1368	40 	390
16 	6051  	120.287	46.413 	30 	360
17 	6062  	101.434	47.7877	10 	370
18 	6079  	84.2   	48.9539	10 	350
19 	6019  	69.832 	48.9634	0  	320
20 	5969  	57.736 	49.7576	0  	350
21 	6126  	47.902 	51.1472	0  	310
22 	6007  	37.352 	49.6839	0  	330
23 	6040  	30.523 	51.1192	0  	300
24 	5915  	24.143 	50.2948	0  	330
25 	5887  	23.033 	51.0064	0  	300
26 	5907  	22.981 	50.7053	0  	280
27 	6080  	23.325 	5

# 結果出力

In [111]:
def decodePlan(individual):
  """配送計画のデコード
  ~args~
  individual : サンプル
  ~returns~
  plan : DAY日分の配送計画、給油対象のリフト
  """
  plan = {}
  for day, supply_plan_in_a_day in enumerate(individual):
    delivery = set()
    supply_lift = []
    for lift_index, lift in enumerate(lifts):
      if supply_plan_in_a_day & (1<<lift_index):
        delivery.add(lift["base"])
        supply_lift.append(lift["id"])
    plan[f"day_{day+1}"] = {"base":delivery, "lift": supply_lift}
  return plan

In [ ]:
## 最適なサンプルの抽出
best_plan = tools.selBest(pop, 1)[0]
## サンプルをデコード
plan = decodePlan(best_plan)

## 表示
print("the best plan is ")
pprint(plan)
print(f"penalty: {best_plan.fitness.values}")